# Task1. 预测是酶还是非酶

> author: Shizhenkun   
> email: zhenkun.shi@tib.cas.cn   
> date: 2021-05-26  

## 任务简介
该任务通过给定蛋白序列，预测该该蛋白是酶还是非酶。本任务所使用的数据集为Sport，对数据集的数据中进行学习，然后对新给定的蛋白序列数据预测是酶还是非酶。


## 数据统计
- 数据源Sprot，酶 219,227 条， 非酶226,539条。
- 将数据集中的所有数据按照时间排序，～90%作为训练集，～10%作为测试集，找到对应时间节点为2009年12月14日。
- 以2009年12月14日为时间节点，之前的数据为训练集，之后的数据为测试集，具体数据集统计如下： 

###  三级以下EC号的算作是酶 

|   Items   | 酶      |   非酶    |合计                               |
|-----------| --------| --------- |----------------------------------|
| 训练集     | 198,692 | 208,391   | 407,083（407,083/453,212=89.82%) |
| 测试集     | 20,535  | 25,594    | 46,129 (46,129/453,212=10.18% )  |


<div style="background-color: lightyellow; border-style: dotted; width:1100px; margin:auto; padding:35px;">


比对结果：
34813(查询有结果的)/46129（测试集总条数）

有查询结果：
酶: 18147/17919; 非酶: 16666/16894   
无查询结果的条数：11316
酶：2616； 非酶：8700
    
    
|baslineName| accuracy 	 |precision(PPV) |	 NPV 	 |	recall |	f1 	 |	 auroc 	 |	 auprc 	 |	 confusion Matrix						|
| ----------| -----------|---------------| --------- | --------|---------|-----------|---------- |------------------------------------------|
|同源比对 <span style="color:red">「无比对结果的全算非酶」 </span> |0.915931    |	0.958946 	  |	 0.888035 | 0.847431|0.899747  | 	 	   |           |tp:17402  fp:745  fn:3133  tn:24849      |
|同源比对 <span style="color:red">「无比对结果的全算酶」 </span>   | 0.784040  |0.679428 |0.968979  |0.974823| 0.800752 | 	 |	| tp:20018  fp:9445  fn:517  tn:16149|
|同源比对 <span style="color:red">「无比对结果的随机猜测1/2算酶，1/2非酶」 </span>   | 0.848685  |0.784351| 0.917467  |0.910397|0.842686| 	| 	 	| tp:18695  fp:5140  fn:1840  tn:20454|
|同源比对 <span style="color:red">「只计算有比对结果的」 </span>	|0.963749  	|0.958946 		   |0.968979 | 	0.971148 |0.965009	|	 	 	| 	| tp:17402  fp:745  fn:517  tn:16149|
|DeepEC	 	| 0.848013  |	0.979370 	|	 0.790151| 0.672754|0.797610  |		 	 |           |tp:13815  fp:291  fn:6720  tn:25303|
|集成XGBOOST | 0.935615  |	0.919272 |	 0.949210|  0.937716	 |0.928403		 |	 |	 |	 tp:19256  fp:1691  fn:1279  tn:23903|
|集成dt 	 |0.912658  	|0.881448 	| 0.940228  |	0.928707	 |0.904460		 |	 |	 | tp:19071  fp:2565  fn:1464  tn:23029|
    
</div>

</br></br>

###  三级以下EC号的扔掉，不包括到数据集中 
    
|    Items    | 酶       |   非酶    |合计                               |
| ------------ | --------| --------- |----------------------------------|
| 训练集        | 198,692 | 202,602   | 401,294（401,294/445,798=90.02%)  |
| 测试集        | 20,535  | 23,969    | 44,504 (44,504/445,798= 9.98% )  |


<div style="background-color: lightgreen; border-style: dotted; width:1100px; margin:auto; padding:35px;">
    

比对结果：
33514(查询有结果的)/44504（测试集总条数）

有查询结果：
酶: 17551/17854; 非酶: 15344/15660   
无查询结果的条数：10990
酶：2681； 非酶：10990
    
    
    
|baslineName| accuracy 	 |precision(PPV) |	 NPV 	 |	recall |	f1 	 |	 auroc 	 |	 auprc 	 |	 confusion Matrix						|
| ----------| -----------|---------------| --------- | --------|---------|-----------|---------- |------------------------------------------|
|同源比对 <span style="color:red">「无比对结果的全算非酶」 </span>  |0.925849  	|0.982314 	|0.887975  |0.854687 | 0.914067	| 	| 	| 	 tp:17551  fp:316  fn:2984  tn:23653|
|同源比对 <span style="color:red">「无比对结果的全算酶」 </span>   | 0.799389 |0.701112   | 0.980635  | 0.985245	| 0.819242	|	 |	| 	  tp:20232  fp:8625  fn:303  tn:15344|
|同源比对 <span style="color:red">「无比对结果的随机猜测1/2算酶，1/2非酶」 </span>   | 0.864394|  	0.810599| 	0.923731|  	0.921402|	 0.862456|	|	| tp:18921  fp:4421  fn:1614  tn:19548|
|同源比对 <span style="color:red">「只计算有比对结果的」 </span>	|0.981530  	|0.982314|0.980635 | 	0.983029	| 0.982671|     |		| 	 tp:17551  fp:316  fn:303  tn:15344|
|同源比对<span style="color:red">「无比对结果的算错误」 </span>  	| 0.739147  |0.751263| 0.725759|  	0.751681	| 0.751472|		 |	 	| 	 tp:17551  fp:5811  fn:5798  tn:15344|
|DeepEC	 	 | 0.842913  |	0.980761  |0.779078  |	0.672754 | 0.798071	| 	 |	 |	 tp:13815  fp:271  fn:6720  tn:23698|
|ECPred	 	 |0.898593   |0.836455 	  | 0.970088 |0.969856	 |0.898230	|	 |	 |	 tp:19916  fp:3894  fn:619  tn:20075|
|集成lr 	 	|0.948454  	|0.941863 	|0.954155  	|0.946725	 |0.944288	|	| 	| 	 tp:19441  fp:1200  fn:1094  tn:22769|
|集成xg 	 	|0.952431  	|0.947691 	|0.956505  	|0.949306	 |0.948498	|	| 	| 	 tp:19494  fp:1076  fn:1041  tn:22893|
|集成dt 	 	|0.926816  	|0.904827 	|0.947073  	|0.940297	 |0.922221	|	| 	| 	 tp:19309  fp:2031  fn:1226  tn:21938|
|集成rf 	 	|0.952746  	|0.954214 	|0.951516  	|0.942829	 |0.948487	|	| 	| 	 tp:19361  fp:929   fn:1174  tn:23040|
|集成gbdt 	|0.951488  	|0.949863 	|0.952865  	|0.944729	 |0.947289	|	| 	| 	 tp:19400  fp:1024  fn:1135  tn:2294 |   
</div>









## 实验方法

- 同源比对：使用训练集建立比对库，然后将测试集与之比对，取最优的比对结果，比对结果的（酶/非酶）属性当作测试集的测试结果
- 传统机器学习方法
- 深度学习方法



## 实验结果

### baselines




<div style="background-color: lightyellow; border-style: dotted; width:1100px; margin:auto; padding:35px;">

###  三级以下EC号的算作是酶     
    
### 机器学习 + onehot


|baslineName| accuracy 	 |precision(PPV) |	 NPV 	 |	recall |	f1 	 |	 auroc 	 |	 auprc 	 |	 confusion Matrix						|
| ----------| -----------|---------------| --------- | --------|---------|-----------|---------- |------------------------------------------|
| lr 		|	0.609676 |	0.560377 	 |	0.680254 |0.715023 |0.628324 |	0.661484 |	0.568861 |	 tp: 14683 fp: 11519 fn: 5852 tn: 12450	|
| xg 		|	0.665446 |	0.643591 	 |	0.682740 |0.616168 |0.629581 |	0.742674 |	0.684483 |	 tp: 12653 fp: 7007 fn: 7882 tn: 16962	|
| dt 		|	0.602418 |	0.570965 	 |	0.628129 |0.556562 |0.563671 |	0.599133 |	0.522388 |	 tp: 11429 fp: 8588 fn: 9106 tn: 15381	|
| rf 		|	0.666479 |	0.624934 	 |	0.710044 |0.693255 |0.657324 |	0.745132 |	0.689971 |	 tp: 14236 fp: 8544 fn: 6299 tn: 15425	|
| gbdt  	| 	0.624079 |	0.569351 	 |	0.712026 |0.760604 |0.651226 |	0.695638 |	0.628716 |	 tp: 15619 fp: 11814 fn: 4916 tn: 12155	|


### 机器学习 + unirep

|baslineName| accuracy 	 |precision(PPV) |	 NPV 	 |	recall |	f1 	 |	 auroc 	 |	 auprc 	 |	 confusion Matrix					  |
| ----------| -----------|---------------| --------- | --------|---------|-----------|---------- |----------------------------------------|
|lr 	    |0.826757 	 |	0.849132 	 |	0.811034 |0.759484 |0.801810 |	0.901500 |	0.892720 |	 tp: 15596 fp: 2771 fn: 4939 tn: 21198|
|xg 	    |0.860260	 |	0.883725 	 |	0.843327 |0.802776 |0.841308 |	0.934751 |	0.927646 |	 tp: 16485 fp: 2169 fn: 4050 tn: 21800|
|dt 	    |0.772290 	 |	0.767612 	 |	0.775916 |0.726418 |0.746447 |	0.769004 |	0.683843 |	 tp: 14917 fp: 4516 fn: 5618 tn: 19453|
|rf 	    |0.851070    |	0.903031 	 |	0.818172 |0.758705 |0.824600 |	0.932207 |	0.922548 |	 tp: 15580 fp: 1673 fn: 4955 tn: 22296|
|gbdt 		|0.822825 	 |0.864764 		 |0.796054 	 |0.730217 |0.791815 |	0.898601 |	0.889249 |	 tp: 14995 fp: 2345 fn: 5540 tn: 21624|

</div>

</br>


<div style="background-color: lightgreen; border-style: dotted; width:1100px; margin:auto; padding:35px;">

###  三级以下EC号的扔掉，不包括到数据集中     
    
### 机器学习 + onehot


|baslineName| accuracy 	 |precision(PPV) |	 NPV 	 |	recall |	f1 	 |	 auroc 	 |	 auprc 	 |	 confusion Matrix						|
| ----------| -----------|---------------| --------- | --------|---------|-----------|---------- |------------------------------------------|
|lr 		|0.608664 |0.560196 |0.676184 |	0.706745 |	0.624995 |	0.658145 |	0.565398 	| tp: 14513 fp: 11394 fn: 6022 tn: 12575	|
|xg 		|0.671468 |0.649797 |0.688742 |	0.624641 |	0.636971 |	0.744063 |	0.685988 	| tp: 12827 fp: 6913 fn: 7708 tn: 17056		|
|dt 		|0.606260 |0.574821 |0.632220 |	0.563428 |	0.569068 |	0.603192 |	0.525313 	| tp: 11570 fp: 8558 fn: 8965 tn: 15411		|
|rf 		|0.664435 |0.622416 |0.708882 |	0.693402 |	0.655994 |	0.744528 |	0.689728 	| tp: 14239 fp: 8638 fn: 6296 tn: 15331		|
|gbdt 		|0.622910 |0.568463 |0.710208 |	0.758753 |	0.649967 |	0.694911 |	0.631114 	| tp: 15581 fp: 11828 fn: 4954 tn: 12141	|


### 机器学习 + unirep

|baslineName| accuracy 	 |precision(PPV) |	 NPV 	 |	recall |	f1 	 |	 auroc 	 |	 auprc 	 |	 confusion Matrix					  |
| ----------| -----------|---------------| --------- | --------|---------|-----------|---------- |----------------------------------------|
|lr 	|0.825521 	|0.847237 	|0.810231 |	0.758656 |	0.800504 |	0.900743 |	0.891768 |	 tp: 15579 fp: 2809 fn: 4956 tn: 21160|
|xg 	|0.857788 	|0.881841 	|0.840514 |	0.798831 |	0.838286 |	0.933706 |	0.927064 |	 tp: 16404 fp: 2198 fn: 4131 tn: 21771|
|dt 	|0.771346 	|0.766943 	|0.774750 |	0.724665 |	0.745205 |	0.768002 |	0.682822 |	 tp: 14881 fp: 4522 fn: 5654 tn: 19447|
|rf 	|0.849811 	|0.901548 	|0.817075 |	0.757195 |	0.823090 |	0.931437 |	0.922190 |	 tp: 15549 fp: 1698 fn: 4986 tn: 22271|

</div>

## 1. 导入必要的包、定义公共函数

In [1]:
import numpy as np
import pandas as pd
import random
from random import randint
import time
import gzip
import re
import datetime
import sys
import os
from tqdm import tqdm

from functools import reduce
import matplotlib.pyplot as plt

sys.path.append("../../tools/")
import commontools
import funclib
from pyecharts.globals import CurrentConfig, OnlineHostType
CurrentConfig.ONLINE_HOST = OnlineHostType.NOTEBOOK_HOST
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

from pyecharts import options as opts
from pyecharts.charts import Bar
from pyecharts.faker import Faker
from pyecharts.globals import ThemeType

%load_ext autoreload
%autoreload 2

## 2. 加载数据

In [2]:
train = pd.read_hdf('./data/train.h5',key='data')
train.ec_number = train.ec_number.apply(lambda x: x.replace(',', ''))
test = pd.read_hdf('./data/test.h5',key='data')
test.ec_number = test.ec_number.apply(lambda x: x.replace(',', ''))
head = funclib.table_head + ['f'+str(i) for i in range(1, 1901) ]
train.columns = head
test.columns = head

## 3. 同源比对

In [62]:
# 读取比对结果
res_data=funclib.getblast(train,test)

Write finished
Write finished
diamond makedb --in /tmp/train.fasta -d /tmp/train.dmnd
diamond blastp -d /tmp/train.dmnd  -q  /tmp/test.fasta -o /tmp/test_fasta_results.tsv -b5 -c1 -k 1


In [63]:
# 给比对结果添加标签
isEmzyme_dict = {v: k for k,v in zip(train.isemzyme, train.id )} 
res_data['isEmzyme_pred'] = res_data['sseqid'].apply(lambda x: isEmzyme_dict.get(x))

isEmzyme_dict_test = {v: k for k,v in zip(test.isemzyme, test.id )} 
res_data['isEmzyme_groudtruth'] = res_data['id'].apply(lambda x: isEmzyme_dict_test.get(x))

In [64]:

# 获取比对结果
res_data_sub = res_data[['id','isEmzyme_pred']]

#选取测试集的ID与是否是酶2列
test_data_sub=test[['id', 'isemzyme']]
test_data_sub.reset_index(drop=True, inplace=True)

#拼合比对结果到测试集
res = pd.merge(test_data_sub,res_data_sub, on='id', how='left')


In [36]:
# 填充没有比对结果的值为非酶
# res.fillna(False, inplace=True)

# # # 填充没有比对结果的值为酶
# res.fillna(True, inplace=True)

# 填充没有比对结果的值 随机填充
list_enornot=[True, False]
for i in range(len(res[(res.isEmzyme_pred.isnull())])):
    res.fillna(random.choice(list_enornot), inplace=True, limit=1)


In [81]:
# 计算指标
tp = len(res[res.isemzyme & res.isEmzyme_pred])
fp = len(res[(res.isemzyme ==False) & (res.isEmzyme_pred)])
tn = len(res[(res.isemzyme ==False) & (res.isEmzyme_pred ==False)])
fn = len(res[(res.isemzyme ) & (res.isEmzyme_pred == False)])
funclib.caculateMetrix_1('同源比对',tp, fp, tn,fn)

同源比对 	 0.981530  	0.982314 		 0.980635  	0.983029	 0.982671		 	 	 	 tp:17551  fp:316  fn:303  tn:15344


## 4. 机器学习方法预测
### 4.1 one-hot + 机器学习

In [68]:
trainset = train[['id', 'isemzyme','seq', 'seqlength']].reset_index(drop=True)
testset = test[['id', 'isemzyme','seq', 'seqlength']].reset_index(drop=True)

MAX_SEQ_LENGTH = 1500 #定义序列最长的长度
trainset.seq = trainset.seq.map(lambda x : x[0:MAX_SEQ_LENGTH].ljust(MAX_SEQ_LENGTH, 'X'))
testset.seq = testset.seq.map(lambda x : x[0:MAX_SEQ_LENGTH].ljust(MAX_SEQ_LENGTH, 'X'))

In [77]:
f_train = funclib.dna_onehot(trainset) #训练集编码
f_test = funclib.dna_onehot(testset) #测试集编码

train_full = pd.concat([trainset, f_train], axis=1, join='inner' ) #拼合训练集
test_full = pd.concat([testset, f_test], axis=1, join='inner' )    #拼合测试集

X_train = train_full.iloc[:,4:]
X_test = test_full.iloc[:,4:]
Y_train = train_full.isemzyme.astype('int')
Y_test = test_full.isemzyme.astype('int')

X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

funclib.run_baseline(X_train, Y_train, X_test, Y_test)

baslineName 	 accuracy 	 precision(PPV) 	 NPV 		 recall 	 f1 		 auroc 		 auprc 		 confusion Matrix
lr 		0.608664 	0.560196 		0.676184 	0.706745 	0.624995 	0.658145 	0.565398 	 tp: 14513 fp: 11394 fn: 6022 tn: 12575
xg 		0.671468 	0.649797 		0.688742 	0.624641 	0.636971 	0.744063 	0.685988 	 tp: 12827 fp: 6913 fn: 7708 tn: 17056
dt 		0.606260 	0.574821 		0.632220 	0.563428 	0.569068 	0.603192 	0.525313 	 tp: 11570 fp: 8558 fn: 8965 tn: 15411
rf 		0.664435 	0.622416 		0.708882 	0.693402 	0.655994 	0.744528 	0.689728 	 tp: 14239 fp: 8638 fn: 6296 tn: 15331
gbdt 		0.622910 	0.568463 		0.710208 	0.758753 	0.649967 	0.694911 	0.631114 	 tp: 15581 fp: 11828 fn: 4954 tn: 12141


### 5. Unirep + 机器学习

In [3]:
X_train = train.iloc[:,12:]
X_test = test.iloc[:,12:]
Y_train = train.iloc[:,2].astype('int')
Y_test = test.iloc[:,2].astype('int')
X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train).flatten()
Y_test = np.array(Y_test).flatten()

funclib.run_baseline(X_train, Y_train, X_test, Y_test)

baslineName 	 accuracy 	 precision(PPV) 	 NPV 		 recall 	 f1 		 auroc 		 auprc 		 confusion Matrix
lr 		0.825521 	0.847237 		0.810231 	0.758656 	0.800504 	0.900743 	0.891768 	 tp: 15579 fp: 2809 fn: 4956 tn: 21160
xg 		0.857788 	0.881841 		0.840514 	0.798831 	0.838286 	0.933706 	0.927064 	 tp: 16404 fp: 2198 fn: 4131 tn: 21771
dt 		0.769167 	0.763209 		0.773810 	0.724519 	0.743361 	0.765969 	0.680072 	 tp: 14878 fp: 4616 fn: 5657 tn: 19353
rf 		0.849811 	0.901548 		0.817075 	0.757195 	0.823090 	0.931437 	0.922190 	 tp: 15549 fp: 1698 fn: 4986 tn: 22271
gbdt 		0.822083 	0.862578 		0.796089 	0.730850 	0.791269 	0.898090 	0.888768 	 tp: 15008 fp: 2391 fn: 5527 tn: 21578


### 6. DeepEC

In [5]:
funclib.table2fasta(test, './data/deepec.fasta')
! python ../../baselines/deepec/deepec.py -i ./data/deepec.fasta -o ./data/deepec/

Write finished
2021-06-19 08:55:00.721087: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-06-19 09:16:38.573926: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-06-19 09:16:38.674269: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:81:00.0 name: GeForce RTX 3080 computeCapability: 8.6
coreClock: 1.8GHz coreCount: 68 deviceMemorySize: 9.78GiB deviceMemoryBandwidth: 707.88GiB/s
2021-06-19 09:16:38.675040: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:c1:00.0 name: GeForce RTX 3080 computeCapability: 8.6
coreClock: 1.8GHz coreCount: 68 deviceMemorySize: 9.78GiB deviceMemoryBandwidth: 707.88GiB/s
2021-06-19 09:16:38.675775: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 2 with properties: 
pciBusID: 0000:c2:00

In [72]:
res_deepec = pd.read_csv('./data/deepec/DeepEC_Result.txt', header=0, sep='\t')
test_results_df=test[['id', 'name', 'isemzyme']]
isEmzyme_dict = {v: k for k,v in zip(test.isemzyme, test.id )} 
with pd.option_context('mode.chained_assignment', None):
    test_results_df['pred'] = np.isin(np.array(test.id), np.array(res_deepec['Query ID']), invert=False)
test_results_df.reset_index(drop=True)

tp = len(test_results_df[test_results_df.isemzyme & test_results_df.pred])
fp = len(test_results_df[(test_results_df.isemzyme ==False) & (test_results_df.pred)]) 
tn = len(test_results_df[(test_results_df.isemzyme ==False) & (test_results_df.pred ==False)])
fn = len(test_results_df[(test_results_df.isemzyme ) & (test_results_df.pred == False)])
funclib.caculateMetrix_1('DeepEC\t',tp, fp, tn,fn)

DeepEC	 	 0.842913  	0.980761 		 0.779078  	0.672754	 0.798071		 	 	 	 tp:13815  fp:271  fn:6720  tn:23698


### 8. ECPred

In [24]:
## 读取结果
ecpred_results = pd.read_csv('./results/ecpred_results_merge.tsv', sep='\t', header=0) #读入文件
# ecpred_sup_results = pd.read_csv('./results/ecpred_sup_results.tsv', sep='\t', header=0)
# ec_dict = {v: k for k,v in zip(test.ec_number, test.id )} 
# ecpred_results['ec_groundtruth'] = ecpred_results.apply(lambda x: ec_dict.get(x['Protein ID']), axis=1)
# ecpred_results['iscorrect'] = ecpred_results.apply(lambda x: x['EC Number'] == x.ec_groundtruth, axis=1)

# ecpred_results_final = pd.concat([ecpred_results, ecpred_sup_results], axis=0, ignore_index = True)
# ecpred_results_final.drop_duplicates(subset=['Protein ID'], keep='first', inplace = True)
# ecpred_results_final.reset_index(drop =True, inplace=True)
# ecpred_results_final.to_csv('./results/ecpred_results_merge.tsv', sep='\t', header=1)

In [61]:
# 填充预测结果
ecpred_results['isemzyme'] = True
with pd.option_context('mode.chained_assignment', None):
    ecpred_results.isemzyme[ecpred_results['EC Number']=='non Enzyme'] = False
    ecpred_results.isemzyme[ecpred_results['EC Number']!='non Enzyme'] = True

# 提取测试集项目
test_results_df=test[['id', 'name', 'isemzyme']]
isEmzyme_pred_dict = {v: k for k,v in zip(ecpred_results.isemzyme, ecpred_results['Protein ID'] )} 

# 拼合测试集与预测结果
with pd.option_context('mode.chained_assignment', None):
    test_results_df['pred']=test_results_df.id.apply(lambda x: isEmzyme_pred_dict.get(x))
    test_results_df['pred'][test_results_df['pred'].isnull()] = False
    
# 计算指标
tp = len(test_results_df[test_results_df.isemzyme & test_results_df.pred])
fp = len(test_results_df[(test_results_df.isemzyme ==False) & (test_results_df.pred)])
tn = len(test_results_df[(test_results_df.isemzyme ==False) & (test_results_df.pred ==False)])
fn = len(test_results_df[(test_results_df.isemzyme ) & (test_results_df.pred == False)])
print('baslineName', '\t', 'accuracy','\t', 'precision(PPV) \t NPV \t\t', 'recall','\t', 'f1', '\t\t', 'auroc','\t\t', 'auprc', '\t\t confusion Matrix')
funclib.caculateMetrix_1('ECPred\t',tp, fp, tn,fn)

baslineName 	 accuracy 	 precision(PPV) 	 NPV 		 recall 	 f1 		 auroc 		 auprc 		 confusion Matrix
ECPred	 	 0.898593  	0.836455 		 0.970088  	0.969856	 0.898230		 	 	 	 tp:19916  fp:3894  fn:619  tn:20075


## 9. 集成模型

In [516]:
res_data=funclib.getblast(train,test)
funclib.run_integrated(res_data, train, test)